In [3]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
import datetime
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report
pd.set_option('display.max_columns', None)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [4]:
df=pd.read_excel('◎位置情報と対応(水道水) 基準満たしているか否か.xlsx')

In [12]:
df.isna().sum()

NAME                   0
Rowid                  0
XY水道水位置情報              0
X                      0
Y                      0
up_down                0
curvature              0
inclination            0
tilt_direction         0
altitude               0
disto_river            2
disto_stations         0
disto_mainroad         0
disto_syorizyo         0
supply_hours           0
no_water_days          0
total_population       0
population_served      0
popu-served            0
number_taps            0
pipelength             0
pipelength_per_pipe    0
served/pipes           0
(popu-served)/pipes    0
oldest_pipe_age        0
ST                     0
RSF                    0
FL                     0
PF                     0
RF                     0
EColi.                 0
dtype: int64

In [5]:
name=df['NAME'].unique()

In [6]:
name

array(['Mahendranagar', 'Dhangadhi', 'Nepalgunj', 'Krishnanagar',
       'Bahadurganj', 'Taulihawa', 'Butwal', 'Bhairahawa', 'Hemja',
       'Pokhara', 'Bharatpur', 'Birgunj', 'Kalaiya', 'Hetauda',
       'Janakpur', 'KTM', 'Gaur', 'Banepa', 'Malangwa', 'Gaushala',
       'Jaleshwor', 'Lahan', 'Rajbiraj', 'Dharan', 'Biratnagar',
       'Bhadrapur'], dtype=object)

In [7]:
ratio=pd.DataFrame([1, 1, 1, 1,
                          1, 1, 0.87, 1, 0,
                          0.45, 1, 1, 1, 0.76,
                          1, 0.62, 1, 0, 1, 1,
                          1, 1, 1, 0.625, 1,
                          1], index=name, columns=['ratio'])

In [8]:
ratio=ratio.reset_index()
ratio=ratio.rename(columns={'index':'NAME'})

In [9]:
DF=pd.merge(df, ratio, on='NAME', how='inner')

In [11]:
DF.to_csv('DATA.csv')

In [10]:
DF

,NAME,Rowid,XY水道水位置情報,X,Y,up_down,curvature,inclination,tilt_direction,altitude,disto_river,disto_stations,disto_mainroad,disto_syorizyo,supply_hours,no_water_days,total_population,population_served,popu-served,number_taps,pipelength,pipelength_per_pipe,served/pipes,(popu-served)/pipes,oldest_pipe_age,ST,RSF,FL,PF,RF,EColi.,ratio
0,Mahendranagar,4,192,80.1732,28.967150,61,864000000,89.976921,244.983109,221,0.008422,0.118698,0.002436,704.208459,9.0,56.78,48936,11298,37638,2002,30.957,0.015463,364.957845,1215.815486,40,0,1,0,0,0,0,1.0
1,Mahendranagar,2,191,ｓ,28.974328,25,432000000,89.980904,36.869896,223,0.000505,0.114137,0.000863,841.848117,9.0,56.78,48936,11298,37638,2002,30.957,0.015463,364.957845,1215.815486,40,0,1,0,0,0,1,1.0
2,Mahendranagar,3,188,ｓｓ,28.958797,0,-288000000,89.947029,123.690071,214,0.002185,0.129072,0.006192,1017.720385,9.0,56.78,48936,11298,37638,2002,30.957,0.015463,364.957845,1215.815486,40,0,1,0,0,0,0,1.0
3,Mahendranagar,1,190,80.1867,28.971595,0,-432000000,89.945984,135.000000,217,0.002295,0.122961,0.001804,737.567540,9.0,56.78,48936,11298,37638,2002,30.957,0.015463,364.957845,1215.815486,40,0,1,0,0,0,0,1.0
4,Dhangadhi,7,197,80.5655,28.705933,177,-432000000,89.955597,305.537689,181,0.003237,0.027089,0.004993,1300.239907,12.0,1.06,36000,26220,9780,4469,50.000,0.011188,524.400000,195.600000,40,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,Bhadrapur,213,148,88.0746,26.545469,0,864000000,89.962547,168.690063,92,0.004671,0.042181,0.017736,4431.704313,9.0,0.66,22000,12062,9938,1822,48.700,0.026729,247.679671,204.065708,46,1,1,0,0,0,1,1.0
216,Bhadrapur,208,152,88.0793,26.561851,171,-720000000,89.914589,296.565063,94,0.003045,0.048199,0.001982,4599.404390,9.0,0.66,22000,12062,9938,1822,48.700,0.026729,247.679671,204.065708,46,1,1,0,0,0,1,1.0
217,Bhadrapur,207,151,88.08,26.558706,35,576000000,89.947029,236.309937,93,0.000700,0.045488,0.004072,4664.033414,9.0,0.66,22000,12062,9938,1822,48.700,0.026729,247.679671,204.065708,46,1,1,0,0,0,1,1.0
218,Bhadrapur,215,149,88.0878,26.539138,0,-144000000,89.729904,135.000000,87,0.003060,0.027479,0.004462,5921.463636,9.0,0.66,22000,12062,9938,1822,48.700,0.026729,247.679671,204.065708,46,1,1,0,0,0,1,1.0
